In [3]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import box
from shapely.geometry import Point

# ----- 1. Load and prepare the GISCO grid data -----
# Read the grid data from a parquet file (as in your Gisco.py)
df = pd.read_parquet("data/grid_10km.parquet")

# Create grid cell geometries from lower-left corner coordinates.
def create_geometry(df, resolution_km):
    cell_size = resolution_km * 1000  # convert km to meters
    df['geometry'] = df.apply(lambda row: box(row['X_LLC'], row['Y_LLC'],
                                              row['X_LLC'] + cell_size, row['Y_LLC'] + cell_size), axis=1)
    return gpd.GeoDataFrame(df, geometry='geometry')

# Here we assume a 10 km resolution (adjust if needed)
grid_gdf = create_geometry(df, resolution_km=10)
grid_gdf = grid_gdf.set_crs(epsg=3035)  # Set to the projected CRS as used in your Gisco.py

# ----- 2. Load and convert the ESA World Cover data -----
# Open the NetCDF file (generated from your Worldcover.py script)
ds = xr.open_dataset("data/ESAWorldCover.nc")

# Assume the world cover classification is in a variable called "MAP".
# Convert the xarray Dataset to a DataFrame.
wc_df = ds[["MAP"]].to_dataframe().reset_index()  # 'x' and 'y' become columns

# Create a GeoDataFrame for the world cover points using the 'x' and 'y' coordinates.
gdf_wc = gpd.GeoDataFrame(wc_df, geometry=gpd.points_from_xy(wc_df['x'], wc_df['y']))

# Set the CRS.
# In your Worldcover.py, the coordinates are used as lon/lat for Folium so they are EPSG:4326.
gdf_wc = gdf_wc.set_crs(epsg=4326)
# Reproject world cover points to match the grid CRS (EPSG:3035)
gdf_wc = gdf_wc.to_crs(epsg=3035)

# ----- 3. Spatial join and aggregation -----
# Perform a spatial join: assign each world cover point to the grid cell that contains it.
# The "op" parameter 'within' ensures points falling inside a polygon are matched.
joined = gpd.sjoin( gdf_wc, grid_gdf, how="inner", predicate="within")

# For each grid cell (identified by, e.g., "GRD_ID"), choose the most common world cover value.
# This is a proxy for the cover with the largest area in the cell.
dominant_cover = joined.groupby("GRD_ID")["MAP"].agg(lambda x: x.value_counts().idxmax()).reset_index()
dominant_cover.columns = ["GRD_ID", "dominant_cover"]

# Merge the dominant cover info back into the grid GeoDataFrame.
grid_gdf = grid_gdf.merge(dominant_cover, on="GRD_ID", how="inner")


In [4]:
import folium

# Définition de la fonction de style pour chaque entité
def style_function(feature):
    # Ici, on suppose que le champ qui contient la valeur s'appelle 'valeur'
    if feature['properties'].get('dominant_cover') == 40:
        return {
            'fillColor': 'red',  # Couleur de remplissage si la valeur est 40
            'color': 'red',      # Couleur de la bordure
            'weight': 2,
            'fillOpacity': 0.6
        }
    else:
        return {
            'fillColor': 'blue',  # Couleur par défaut
            'color': 'blue',
            'weight': 1,
            'fillOpacity': 0.4
        }

# Création de la carte
m = folium.Map(location=[49.85, 6.15], zoom_start=10)

# Ajout du GeoJson avec la fonction de style personnalisée
folium.GeoJson(grid_gdf, style_function=style_function).add_to(m)

m


In [5]:
joined

,t,y,x,MAP,geometry,index_right,DIST_BORD,TOT_P_2018,TOT_P_2006,GRD_ID,...,NUTS2016_2,NUTS2016_1,NUTS2016_0,LAND_PC,X_LLC,NUTS2021_3,NUTS2021_2,DIST_COAST,NUTS2021_1,NUTS2021_0
0,2021-01-01,51.262146,4.360912,50.0,POINT (3927832.116 3143087.359),37428,9070.639648,3571,5083,CRS3035RES10000mN3140000E3920000,...,BE21-BE23,BE2,BE,88.250000,3920000,BE211-BE236,BE21-BE23,9810.900391,BE2,BE
1,2021-01-01,51.262146,4.360995,50.0,POINT (3927837.913 3143086.911),37428,9070.639648,3571,5083,CRS3035RES10000mN3140000E3920000,...,BE21-BE23,BE2,BE,88.250000,3920000,BE211-BE236,BE21-BE23,9810.900391,BE2,BE
2,2021-01-01,51.262146,4.361079,50.0,POINT (3927843.71 3143086.462),37428,9070.639648,3571,5083,CRS3035RES10000mN3140000E3920000,...,BE21-BE23,BE2,BE,88.250000,3920000,BE211-BE236,BE21-BE23,9810.900391,BE2,BE
3,2021-01-01,51.262146,4.361162,50.0,POINT (3927849.506 3143086.014),37428,9070.639648,3571,5083,CRS3035RES10000mN3140000E3920000,...,BE21-BE23,BE2,BE,88.250000,3920000,BE211-BE236,BE21-BE23,9810.900391,BE2,BE
4,2021-01-01,51.262146,4.361245,50.0,POINT (3927855.303 3143085.565),37428,9070.639648,3571,5083,CRS3035RES10000mN3140000E3920000,...,BE21-BE23,BE2,BE,88.250000,3920000,BE211-BE236,BE21-BE23,9810.900391,BE2,BE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689445,2021-01-01,51.174729,4.494579,40.0,POINT (3936401.622 3132674.318),37085,18735.199219,396502,324323,CRS3035RES10000mN3130000E3930000,...,BE21,BE2,BE,99.699997,3930000,BE211-BE212,BE21,23801.990234,BE2,BE
1689446,2021-01-01,51.174729,4.494662,40.0,POINT (3936407.43 3132673.88),37085,18735.199219,396502,324323,CRS3035RES10000mN3130000E3930000,...,BE21,BE2,BE,99.699997,3930000,BE211-BE212,BE21,23801.990234,BE2,BE
1689447,2021-01-01,51.174729,4.494745,40.0,POINT (3936413.239 3132673.441),37085,18735.199219,396502,324323,CRS3035RES10000mN3130000E3930000,...,BE21,BE2,BE,99.699997,3930000,BE211-BE212,BE21,23801.990234,BE2,BE
1689448,2021-01-01,51.174729,4.494829,40.0,POINT (3936419.048 3132673.002),37085,18735.199219,396502,324323,CRS3035RES10000mN3130000E3930000,...,BE21,BE2,BE,99.699997,3930000,BE211-BE212,BE21,23801.990234,BE2,BE


In [6]:
grid_gdf

,DIST_BORD,TOT_P_2018,TOT_P_2006,GRD_ID,TOT_P_2011,TOT_P_2021,Y_LLC,CNTR_ID,NUTS2016_3,NUTS2016_2,...,NUTS2016_0,LAND_PC,X_LLC,NUTS2021_3,NUTS2021_2,DIST_COAST,NUTS2021_1,NUTS2021_0,geometry,dominant_cover
0,16344.099609,188821,169215,CRS3035RES10000mN3130000E3920000,179742,193306,3130000,BE,BE211-BE236,BE21-BE23,...,BE,99.139999,3920000,BE211-BE236,BE21-BE23,19047.910156,BE2,BE,"POLYGON ((3930000 3130000, 3930000 3140000, 39...",50.0
1,18735.199219,396502,324323,CRS3035RES10000mN3130000E3930000,377795,400661,3130000,BE,BE211-BE212,BE21,...,BE,99.699997,3930000,BE211-BE212,BE21,23801.990234,BE2,BE,"POLYGON ((3940000 3130000, 3940000 3140000, 39...",50.0
2,9070.639648,3571,5083,CRS3035RES10000mN3140000E3920000,3304,3739,3140000,BE,BE211-BE236,BE21-BE23,...,BE,88.250000,3920000,BE211-BE236,BE21-BE23,9810.900391,BE2,BE,"POLYGON ((3930000 3140000, 3930000 3150000, 39...",50.0
3,9109.459961,174835,183281,CRS3035RES10000mN3140000E3930000,169574,176560,3140000,BE,BE211,BE21,...,BE,98.260002,3930000,BE211,BE21,17139.390625,BE2,BE,"POLYGON ((3940000 3140000, 3940000 3150000, 39...",50.0
